<a href="https://colab.research.google.com/github/rudevico/Gachon-AISTUDY/blob/main/11_Cross_validation_and_Grid_serch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [ ]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [ ]:
wine.describe()

,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


In [ ]:
wine['class'].unique()

array([0., 1.])

In [ ]:
# drop. axis=0, refers to row.
# drop. axis=1, refers to column. 즉 class라는 column을 drop하라.
X = wine.drop('class', axis=1).to_numpy()
y = wine['class'].to_numpy()

X[:5]

array([[9.4 , 1.9 , 3.51],
       [9.8 , 2.6 , 3.2 ],
       [9.8 , 2.3 , 3.26],
       [9.8 , 1.9 , 3.16],
       [9.4 , 1.9 , 3.51]])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)

(5197, 3) (1300, 3)


train set을 다시 sub train set과 validation set으로 split하자.

In [ ]:
X_sub_train, X_val, y_sub_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

print(X_sub_train.shape, X_val.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_sub_train, y_sub_train)
print(dt.score(X_sub_train, y_sub_train))
print(dt.score(X_val, y_val))

print(dt)

0.9971133028626413
0.864423076923077
DecisionTreeClassifier(random_state=42)


overfitting이라고 판단할 수 있겠다.  
또한 다음과 같은 문제가 발생한다.

> val set을 만드느라 train_set의 size가 감소했다.  
(본 예제 기준)
* train_set: 0.80 -> sub_train_set: 0.80*0.80=0.64
* test_set: 0.25(default) -> 0.20
* val_set: 0.00 -> 0.80*0.20=0.16

일반적으로 train set의 size가 클 수록, optimize한 model이 만들어진다.  
그렇다고 해서 val_set의 크기를 더 줄이면, validation score가 들쭉날쭉하고 불안정해져서 ***parameter tuning**이 제대로 이루어지지 않을 것이다.  
> *In machine learning, __hyperparameter optimization__(or tuning) is the problem of choosing a set of optimal hyperparameters for a learning algorithm.  
A hyperparameter is a parameter whose value is used to control the learning precess.

* * *

이때 [***k-fold cross validation**](http://mlwiki.org/index.php/Cross-Validation#K-Fold_Cross-Validation)을 사용하면 train set의 크기를 크게 줄이지 않고도 즉, val set을 적은 크기 만큼만 할당해도 편차가 적은 validation을 할 수 있다.  

[manual of sklearn cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)

In [ ]:
from sklearn.model_selection import cross_validate

# default, cv=5. k-fold에서 k의 value를 설정
scores = cross_validate(dt, X_train, y_train, cv=5)
print(scores)
print(type(scores))
print("- validation score:", scores['test_score'])
print("- 평균:", sum(scores['test_score'])/len(scores['test_score']))

{'fit_time': array([0.01617789, 0.01912379, 0.01541567, 0.011482  , 0.01024675]), 'score_time': array([0.004951  , 0.0018177 , 0.00157213, 0.00171924, 0.0016315 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
<class 'dict'>
- validation score: [0.86923077 0.84615385 0.87680462 0.84889317 0.83541867]
- 평균: 0.855300214703487


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold

# default, n_splits=5. shuffle=False
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, X_train, y_train, cv=splitter)

# 위 mean of scores와 값이 다른 것을 확인할 수 있다.
# cause1. n_splits is 5(default) -> 10
# cause2. shuffle is False(default) -> True
print(np.mean(scores['test_score']))

0.8574181117533719


> 추가적으로 설명하자면,
* Regression의 default splitter is `KFold`
* Classification의 default splitter is `StratifiedKFold`

* * *
이제 hyperparameter tuning을 수행하자.

In [ ]:
from sklearn.model_selection import GridSearchCV

# parameter name: search value의 dict
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# default, n_jobs=1. 사용할 CPU 코어 수. -1이면 모든 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [ ]:
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
# 앞서 수행한 GridSearchCV()는 그리드 서치를 통해서 최적의 parameter값을 찾아내고, 그 값으로 fitting한 Model을 리턴함
# 리턴 객체는 gs.best_estimator_에 보관
dt = gs.best_estimator_
print(dt) # hyperparameter tuning까지 된 Model을 확인 가능

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)


In [ ]:
# 따라서 별도의 fitting없이(왜냐하면 GridSearchCV가 이미 수행함) 곧바로 score()하면 된다
print(dt.score(X_train, y_train))

0.9615162593804117


In [ ]:
# grid search를 통해 찾은 optimal parameter는 best_params_ attribute에 보관
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
# 각 parameter에서 수행한 cross validation과 관련된 데이터들은 다음에 보관
print(gs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_min_impurity_decrease', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [ ]:
print(params)
print('======')
print(params['min_impurity_decrease'])
print(gs.cv_results_['mean_fit_time'])
print(gs.cv_results_['std_fit_time'])
print(gs.cv_results_['mean_score_time'])
print(gs.cv_results_['std_score_time'])
print('...이하 생략')
print(gs.cv_results_['split0_test_score'])
print(gs.cv_results_['split1_test_score'])
print(gs.cv_results_['split2_test_score'])
print(gs.cv_results_['split3_test_score'])
print(gs.cv_results_['split4_test_score'])
print('각 col의 sum / 5 = 아래 각 col value')
print(gs.cv_results_['mean_test_score'])
print('...이하 생략')

{'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
[0.01750231 0.01732192 0.01269879 0.01376123 0.01004438]
[0.0032121  0.00289041 0.00472856 0.00435182 0.00405689]
[0.00223894 0.00176373 0.002739   0.00148664 0.00613017]
[1.19202677e-03 4.74513798e-04 2.38504610e-03 6.14320250e-05
 4.60155251e-03]
...이하 생략
[0.86923077 0.87115385 0.86923077 0.86923077 0.86538462]
[0.86826923 0.86346154 0.85961538 0.86346154 0.86923077]
[0.8825794  0.87680462 0.87584216 0.88161694 0.8825794 ]
[0.86717998 0.85466795 0.85081809 0.84889317 0.84985563]
[0.85370549 0.85659288 0.86910491 0.87584216 0.87102984]
각 col의 sum / 5 = 아래 각 col value
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
...이하 생략


In [ ]:
print(type(gs.cv_results_['mean_test_score']))
print(gs.cv_results_['mean_test_score'])
# np.argmax를 통해서 np array type의 argument 중 max value의 index를 리턴
best_index = np.argmax(gs.cv_results_['mean_test_score'])

print(gs.cv_results_['params'])
print(gs.cv_results_['params'][best_index])

<class 'numpy.ndarray'>
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
[{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}]
{'min_impurity_decrease': 0.0001}


이제 조금 더 복잡한 params combination을 `GridSerachCV()`를 통해 탐색하자.

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), # start, stop, step. 따라서 9 elements.
                                                                     # impurity가 minimum 이 만큼은 decrease해야 한다고 지정해주는 값
          'max_depth': range(5, 20, 1), # python built-in function. integer만 사용 가능. 15 elements
          'min_samples_split': range(2, 100, 10) # 10 elements. node를 split하기 위한 min samples 수
          }

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
print(gs)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

(위 cell 설명) GridSearchCV는 default로 5-fold cross validation을 수행한다.  
또한 hyperparameter tuning은 모든 params에 대한 경우의 수를 대입하므로 다음과 같다.  
* 9 elements by `min_impurity_decrease` parameter
* 15 elements by `max_depth` parameter
* 10 elements by `min_samples_split` parameter
* so, $ 9 * 15 * 10 = 1350 $ for hyperparameter tuning
* 이때 위 과정을 5 fold에 대해서 cross validation 하기 때문에 product 5.
* so, $ 1350 * 5 = 6750 $ 개의 model을 만들고 그 중에 가장 optimal한 model을 `gs.best_estimator_`에 리턴한다.

In [ ]:
print(gs.best_params_) # optimal combination of parameters
print(np.max(gs.cv_results_['mean_test_score'])) # optimal model의 (5-fold cross validation에 대한) mean test score

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


* * *
앞에서 `params`를 설정할 때 범위(start ~ stop)와 간격(step)을 일정한 값으로 설정했었다.
```
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
```
그런데 이 경우 다음과 같은 문제가 발생한다.  
* parameter가 numeric type일 때 start, stop, step 값을 미리 정하기 어려울 수 있다.
* parameter가 너무 많을 경우, grid search 시간이 오래 걸릴 수 있다.

> 이때 **Random Search**를 사용해서 문제를 해소할 수 있다.  
**Random Search**는 parameter list를 전달하는 것이 아니라 **Probability Distribution** Object를 전달한다.

In [ ]:
from scipy.stats import uniform, randint
# 주어진 범위 내에서 randint()는 interger, uniform()은 float 값을 고르게 뽑는다.

In [ ]:
rgen = randint(0, 10) # rgen is random generator
rgen.rvs(10)

array([8, 2, 9, 1, 2, 8, 8, 4, 1, 9])

sampling 수가 늘어날수록 각 int count가 평균에 가까워짐을 확인할 수 있다.  
따라서 sampling은 computer resource 내에서 최대한 크게 설정하는 것이 신뢰도가 높겠다.

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 83, 106, 108, 100, 101, 105, 100, 110,  96,  91]))

In [ ]:
ugen = uniform(0, 1) # float
ugen.rvs(10)

array([0.76568863, 0.71679619, 0.92994185, 0.43332654, 0.71411624,
       0.5379085 , 0.82636399, 0.93954426, 0.77968143, 0.66005781])

In [ ]:
# params에는 Class object만 설정함
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25) # 해당 value보다 samples 수가 적으면 leaf node가 될 수 없음
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7da12bce37c0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7da12c1d6020>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7da12bce32b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7da12bce2530>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(gs)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7da12bce37c0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7da12c1d6020>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7da12bce32b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7da12bce2530>},
                   random_state=42)


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))
print(gs.best_estimator_)
print(gs.best_estimator_.score(X_train, y_train))

0.8695428296438884
DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)
0.8928227823744468


In [ ]:
dt = gs.best_estimator_
print(dt.score(X_test, y_test))

0.86
